**Подключение библиотек и скриптов**

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
import warnings
warnings.filterwarnings('ignore')

**Пути к директориям и файлам**

In [8]:
TRAIN_DATASET_PATH = 'data/train.csv'
PREPARED_TRAIN_DATASET_PATH = 'data/train_prepared.csv'
TEST_DATASET_PATH = 'data/test.csv'

### Загрузка данных

**Описание датасета**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [6]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df.tail()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A,189050.289571
9997,5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,159143.805370
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,0.307467,B,A,30,5048,9,325.0,2,5,B,181595.339808
9999,6306,128,1.0,38.666645,21.157874,8.0,7,17.0,1990,0.000000,B,B,27,4798,0,30.0,2,8,B,218714.077615


In [9]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
4995,8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,B,B,36,5992,0,NaN,1,1,B
4996,4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,B,B,1,264,0,NaN,0,1,B
4997,5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,B,B,74,19083,2,NaN,5,15,B
4998,4780,62,2.0,81.305222,NaN,0.0,4,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A
4999,12504,30,2.0,60.555693,NaN,1.0,10,17.0,1977,0.000078,B,B,22,6398,141,1046.0,3,23,B


In [10]:
train_df.shape[1]-1 == test_df.shape[1]

True

### Приведение типов

In [9]:
train_df.dtypes

Id                 int64
DistrictId         int64
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [11]:
#Задание №1 
#привести Id и DistrictId к типу str

### Обзор данных

**Количественные переменные**

In [12]:
#Задание №2
#С помощью .describe получить первичное понимание того, что из себя представляют данные

**Номинативные переменные**

In [13]:
#Задание №3
#С помощью .select_dtypes получить список категориальных признаков
#Также посчитать количество уникальных значений этих категориальных признаков

### Обработка выбросов

In [18]:
#Задание №4
#Обработать выбросы, подсказка: обратить внимание на признаки, связанные с параметрами домов, год постройки
#Выбросы в некоторых признаках можно найти с помощью квантилей.

### Обработка пропусков

In [17]:
#Задание №5
#Посчитать количество пропусков в каждом признаке
#Заменить пропуски медианой

### Построение новых признаков

**OneHotEncoding**

In [ ]:
#Задание №5
#Для категориальных переменных(несвязанных с Id) провести onehotencoding 

Построить новые 2 признака **DistrictSize, IsDistrictLarge**
где DistrictSize = количество уникальных значений DistrictId в train_df(например, DistrictId=35 встречалось в train_df 22 раза, значит DistrictSize=22)
Для этого создайте датафрейм district_size на основе информации из train_df. В датафрейме district_size должны быть 2 столбца: DistrictId, DistrictSize. После смерджить полученный датасет с train_df и сохранить результат в train_df.


IsDistrictLarge = это индикатор(1 или 0) того, что DistrictSize>100 или нет

Для примера возьму значения 1 строки:
Id = 14038; DistrictId = 35; DistrictSize = 22; IsDistrictLarge = 0

In [ ]:
#ваш код

Построение признака **MedPriceByDistrict**

Создайте датафрейм med_price_by_district, который содержит информацию о медианной цене домов в одном районе и с одинаковым количеством комнат. После смерджите med_price_by_district и train_df по 2-м признакам

In [ ]:
#ваш код